In [44]:
import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

import pandas as pd
import xmltodict
import requests
import os
import json
import numpy as np
from datetime import datetime, timedelta
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값

def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     

In [17]:
appid = 1248716281

In [18]:
url = 'https://itunes.apple.com/kr/rss/customerreviews/page=1/id=%i/sortby=mostrecent/xml' % appid
try:
    response = requests.get(url).content.decode('utf8')
    xml = xmltodict.parse(response)
    last_url = [l['@href'] for l in xml['feed']['link'] if (l['@rel'] == 'last')][0]
    last_index = [int(s.replace('page=', '')) for s in last_url.split('/') if ('page=' in s)][0]
    
except Exception as e:
    print (url)
    print ('\tNo Reviews: appid %i' %appid)
    print ('\tException:', e)

In [27]:
result = list()
for idx in range(1, 14+1):
    url = "https://itunes.apple.com/kr/rss/customerreviews/page=%i/id=%i/sortby=mostrecent/xml?urlDesc=/customerreviews/id=%i/sortBy=mostRecent/xml" % (idx, appid, appid)
#     print(url)
    response = requests.get(url).content.decode('utf8')
    try:
        xml = xmltodict.parse(response)
    except Exception as e:
        print ('\tXml Parse Error %s\n\tSkip %s :' %(e, url))
        continue

    try:
        num_reivews= len(xml['feed']['entry'])
    except Exception as e:
        print ('\tNo Entry', e)
        continue

    try:
        xml['feed']['entry'][0]['author']['name']
        single_reviews = False
    except:
        #print ('\tOnly 1 review!!!')
        single_reviews = True
        pass

    if single_reviews:
            result.append({
                'USER': xml['feed']['entry']['author']['name'],
                'DATE': xml['feed']['entry']['updated'],
                'STAR': int(xml['feed']['entry']['im:rating']),
                'LIKE': int(xml['feed']['entry']['im:voteSum']),
                'TITLE': xml['feed']['entry']['title'],
                'REVIEW': xml['feed']['entry']['content'][0]['#text'],
            })
    else:
        for i in range(len(xml['feed']['entry'])):
            result.append({
                'USER': xml['feed']['entry'][i]['author']['name'],
                'DATE': xml['feed']['entry'][i]['updated'],
                'STAR': int(xml['feed']['entry'][i]['im:rating']),
                'LIKE': int(xml['feed']['entry'][i]['im:voteSum']),
                'TITLE': xml['feed']['entry'][i]['title'],
                'REVIEW': xml['feed']['entry'][i]['content'][0]['#text'],
            })

df_ = pd.DataFrame(result)
df_

	Xml Parse Error syntax error: line 1, column 0
	Skip https://itunes.apple.com/kr/rss/customerreviews/page=11/id=1248716281/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1248716281/sortBy=mostRecent/xml :
	Xml Parse Error syntax error: line 1, column 0
	Skip https://itunes.apple.com/kr/rss/customerreviews/page=12/id=1248716281/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1248716281/sortBy=mostRecent/xml :
	Xml Parse Error syntax error: line 1, column 0
	Skip https://itunes.apple.com/kr/rss/customerreviews/page=13/id=1248716281/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1248716281/sortBy=mostRecent/xml :
	Xml Parse Error syntax error: line 1, column 0
	Skip https://itunes.apple.com/kr/rss/customerreviews/page=14/id=1248716281/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1248716281/sortBy=mostRecent/xml :


,USER,DATE,STAR,LIKE,TITLE,REVIEW
0,s1683740,2022-08-26T17:16:56-07:00,4,0,액면분할,액면불한한거 차트에반영시켜주세요.\n갑자기 뚝떨어진거처럼 차트안나오게
1,살아가는데로,2022-08-26T06:34:41-07:00,1,0,뭔 렉이,똥망. 욕먹을 짓을 하네요\n개발자가 주식거래를 미래말고 딴걸로 하게끔 유도하네요
2,정그린3,2022-08-26T03:01:35-07:00,1,0,목록에 주가가 표시가 안되는버그ㅡㅡ,장난하냐?? 앱개발자 정신차려라
3,밤비미,2022-08-25T21:59:57-07:00,1,0,다시돌려주셈,뒤에버전더 쓸수있게해주던가\n현재가랑 주문이랑 다른 메뉴 사용시\n밑에 메뉴판 안없...
4,투자왕ㅎ,2022-08-25T01:42:07-07:00,1,0,별로,예전 어플이 훨씬 나은듯 보기도 쉽지 않고 뭐 이리 복잡한지 다른 증권사로 옮기렵니다.
...,...,...,...,...,...,...
495,새벽인,2021-04-14T09:07:02-07:00,1,0,신분증 촬영하는데 어지간하면 촬영 되어야지,이건 뭐 아주 정교하게 0.001mm 오차 범위도 허용하지 않는 촬영을 해야함\n짜...
496,다따기태용찡,2021-04-13T17:04:46-07:00,1,0,수수료 때문인지는 모르겠는데,수수료 때문인지는 모르겠는데 83700원에 주문했는데 84110원으로 주문 됐다고 ...
497,근면이,2021-04-13T01:25:44-07:00,1,0,토론실 안 뜸,토론실을 볼 수 있게 해주세요
498,가하라z,2021-04-13T00:42:13-07:00,3,0,차트에서 외국인/기관 누적 순매수 않보임,잘 동작 했는데 갑자기 하나가 않보이거나 둘다 표시 않됩니다. 종목 마다 보이는게 ...


In [41]:
df_['DATETIME'] = df_['DATE'].str.replace("-","").str.replace("T","-").str.replace(":","").str[:15]
df_['VIEWS'] = np.nan
df_['LIKES'] = df_['LIKE']
df_['TITLE'] =  df_['TITLE']
df_['CONTENT'] = df_['REVIEW']
df_['WRITER'] = df_['USER']

df_['SOURCE'] = "Appstore_m.Stock"
df_['RELATED_ITEM'] = "미래에셋증권"
df_['POSITIVITY'] = np.nan
for i in range(len(df_)):
    df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    
df_['OTHERS'] = df_[['STAR']].to_dict('records')
df_['OTHERS'] = df_['OTHERS'].astype(str)
df_['NUM_REPLY'] = np.nan
df_['CRAWLING_DATETIME'] = (datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')
df_['URL'] =  np.nan
df_ttl = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,Appstore_m.Stock,NaN,20220826-171656,액면분할,액면불한한거 차트에반영시켜주세요.\n갑자기 뚝떨어진거처럼 차트안나오게,0.056825,NaN,NaN,20220830-092343,{'STAR': 4}
1,Appstore_m.Stock,NaN,20220826-063441,뭔 렉이,똥망. 욕먹을 짓을 하네요\n개발자가 주식거래를 미래말고 딴걸로 하게끔 유도하네요,0.003060,NaN,NaN,20220830-092343,{'STAR': 1}
2,Appstore_m.Stock,NaN,20220826-030135,목록에 주가가 표시가 안되는버그ㅡㅡ,장난하냐?? 앱개발자 정신차려라,0.001791,NaN,NaN,20220830-092343,{'STAR': 1}
3,Appstore_m.Stock,NaN,20220825-215957,다시돌려주셈,뒤에버전더 쓸수있게해주던가\n현재가랑 주문이랑 다른 메뉴 사용시\n밑에 메뉴판 안없...,0.008585,NaN,NaN,20220830-092343,{'STAR': 1}
4,Appstore_m.Stock,NaN,20220825-014207,별로,예전 어플이 훨씬 나은듯 보기도 쉽지 않고 뭐 이리 복잡한지 다른 증권사로 옮기렵니다.,0.011261,NaN,NaN,20220830-092343,{'STAR': 1}
...,...,...,...,...,...,...,...,...,...,...
495,Appstore_m.Stock,NaN,20210414-090702,신분증 촬영하는데 어지간하면 촬영 되어야지,이건 뭐 아주 정교하게 0.001mm 오차 범위도 허용하지 않는 촬영을 해야함\n짜...,0.048408,NaN,NaN,20220830-092343,{'STAR': 1}
496,Appstore_m.Stock,NaN,20210413-170446,수수료 때문인지는 모르겠는데,수수료 때문인지는 모르겠는데 83700원에 주문했는데 84110원으로 주문 됐다고 ...,0.010897,NaN,NaN,20220830-092343,{'STAR': 1}
497,Appstore_m.Stock,NaN,20210413-012544,토론실 안 뜸,토론실을 볼 수 있게 해주세요,0.007500,NaN,NaN,20220830-092343,{'STAR': 1}
498,Appstore_m.Stock,NaN,20210413-004213,차트에서 외국인/기관 누적 순매수 않보임,잘 동작 했는데 갑자기 하나가 않보이거나 둘다 표시 않됩니다. 종목 마다 보이는게 ...,0.072833,NaN,NaN,20220830-092343,{'STAR': 3}


In [ ]:
df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = 'Appstore_m.Stock' ORDER BY DATETIME desc",conn)
df_already


In [49]:
df_ttl = df_ttl[df_ttl['DATETIME'] > df_already['DATETIME'].max()].reset_index()

In [50]:
if len(df_ttl) >0:

    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl
    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')  , " SUCCESS  : ",len(df_ttl))

else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " NO DATA TO UPDATE")


20220830-092602  NO DATA TO UPDATE


,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,Appstore_m.Stock,None,20220826-171656,액면분할,액면불한한거 차트에반영시켜주세요.\n갑자기 뚝떨어진거처럼 차트안나오게,0.056825,None,None,20220830-092343,{'STAR': 4}
1,Appstore_m.Stock,None,20220826-063441,뭔 렉이,똥망. 욕먹을 짓을 하네요\n개발자가 주식거래를 미래말고 딴걸로 하게끔 유도하네요,0.003060,None,None,20220830-092343,{'STAR': 1}
2,Appstore_m.Stock,None,20220826-030135,목록에 주가가 표시가 안되는버그ㅡㅡ,장난하냐?? 앱개발자 정신차려라,0.001791,None,None,20220830-092343,{'STAR': 1}
3,Appstore_m.Stock,None,20220825-215957,다시돌려주셈,뒤에버전더 쓸수있게해주던가\n현재가랑 주문이랑 다른 메뉴 사용시\n밑에 메뉴판 안없...,0.008585,None,None,20220830-092343,{'STAR': 1}
4,Appstore_m.Stock,None,20220825-014207,별로,예전 어플이 훨씬 나은듯 보기도 쉽지 않고 뭐 이리 복잡한지 다른 증권사로 옮기렵니다.,0.011261,None,None,20220830-092343,{'STAR': 1}
...,...,...,...,...,...,...,...,...,...,...
495,Appstore_m.Stock,None,20210414-090702,신분증 촬영하는데 어지간하면 촬영 되어야지,이건 뭐 아주 정교하게 0.001mm 오차 범위도 허용하지 않는 촬영을 해야함\n짜...,0.048408,None,None,20220830-092343,{'STAR': 1}
496,Appstore_m.Stock,None,20210413-170446,수수료 때문인지는 모르겠는데,수수료 때문인지는 모르겠는데 83700원에 주문했는데 84110원으로 주문 됐다고 ...,0.010897,None,None,20220830-092343,{'STAR': 1}
497,Appstore_m.Stock,None,20210413-012544,토론실 안 뜸,토론실을 볼 수 있게 해주세요,0.007500,None,None,20220830-092343,{'STAR': 1}
498,Appstore_m.Stock,None,20210413-004213,차트에서 외국인/기관 누적 순매수 않보임,잘 동작 했는데 갑자기 하나가 않보이거나 둘다 표시 않됩니다. 종목 마다 보이는게 ...,0.072833,None,None,20220830-092343,{'STAR': 3}
